In [ ]:
import pandas as pd
from huggingface_hub import hf_hub_download
from streaming import MDSWriter
from streaming.base.format.mds.encodings import Encoding, _encodings
from streaming import LocalDataset
import numpy as np
from tqdm import tqdm
import os
from transformers import AutoTokenizer

class UInt32(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.uint32)

_encodings['uint32'] = UInt32

columns = {
    'input_ids': 'uint32',
    'position_ids': 'uint32',
    'attention_mask': 'uint32',
}
hashes = 'sha1', 'xxh64'

In [8]:
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen3-1.7B-Base')

In [3]:
hf_hub_download(
    repo_id="karpathy/fineweb-edu-100b-shuffle", 
    filename="shard_01822.parquet", 
    repo_type="dataset",
    local_dir="./"
)

'shard_01822.parquet'

In [6]:
texts = pd.read_parquet('shard_01822.parquet')['text'].tolist()
len(texts)

53248

In [20]:
def collator(batch, batch_position_ids):
    input_ids = []
    position_ids = []
    masks = []
    for i in range(len(batch)):
        l = len(batch[i])
        input_ids.extend(batch[i])
        position_ids.extend(batch_position_ids[i])
        masks.append(l)
    
    return {
        'input_ids': np.array(input_ids).astype(np.uint32),
        'position_ids': np.array(position_ids).astype(np.uint32),
        'attention_mask': np.array(masks).astype(np.uint32),
    }

def slice_and_balance(nested_list, size):
    first = []
    balance = []
    current_size = 0

    for sublist in nested_list:
        if current_size < size:
            remaining_space = size - current_size
            if len(sublist) <= remaining_space:
                first.append(sublist)
                current_size += len(sublist)
            else:
                first.append(sublist[:remaining_space])
                balance.append(sublist[remaining_space:])
                current_size = size
        else:
            balance.append(sublist)
    
    return first, balance

def loop(rows, block_size = 4096, folder = 'tokenized-4k-qwen'):
    rows, index = rows
    out_root = f'{folder}/tokenized-{index}'
    os.system(f'rm -rf {out_root}')
    count = 0
    temp = []
    position_ids = []
    last_block, last_position_block = None, None
    with MDSWriter(out=out_root, columns=columns, compression=None, hashes=hashes) as out:
        for row in tqdm(rows):
            outputs = tokenizer(row, add_special_tokens = False)
            temp.append(outputs['input_ids'])
            position_ids.append(range(len(outputs['input_ids'])))
            count += len(outputs['input_ids'])
            while count >= block_size:
                block, temp = slice_and_balance(temp, block_size)
                block_position, position_ids = slice_and_balance(position_ids, block_size)
                count = count - block_size
                o = collator(block, block_position)
                last_block = block
                last_position_block = block_position
                out.write(o)
                
        block, _ = slice_and_balance(last_block, block_size - count)
        block_position, _ = slice_and_balance(last_position_block, block_size - count)

        block.extend(temp)
        block_position.extend(position_ids)

        o = collator(block, block_position)
        if len(o['input_ids']) == block_size:
            out.write(o)
            return o

In [28]:
loop((texts, 0))

100%|██████████| 53248/53248 [01:54<00:00, 463.19it/s]


{'input_ids': array([   15,    15,  1251, ...,    13, 85016, 53655], dtype=uint32),
 'position_ids': array([ 596,  597,  598, ..., 2401, 2402, 2403], dtype=uint32),
 'attention_mask': array([1920,  149, 1650,  377], dtype=uint32)}

In [33]:
dataset = LocalDataset('tokenized-4k-qwen/tokenized-0')
len(dataset), dataset[0]['input_ids'].shape

(13298, (4096,))

In [ ]:
!hf upload Scicom-intl/mosaic-dummy-fineweb-edu-100b-shuffle tokenized-4k-qwen/tokenized-0 --repo-type=dataset